# Введение в NLP, часть 2


In [1]:
!pip install -U pip
!pip install transformers datasets torch seqeval evaluate

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 55.1 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=4371616dfba78d67330a0fd128d0630836861a929ee13845091984eb2795a181
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 


## NER c BERT (25 баллов)

1. Взять датасет из предыдущего ДЗ и обучить на нём BERT.
2. Обучить BERT на подготовленном датасете
3. Оценить результат, сравнить с моделью из первого ДЗ

### Подготовка данных (5 баллов)

Подумать о:
1) Как subword токенизация повлияет на BIO раззметку?
2) Что делать с `[CLS]` и `[SEP]` токенами? (Проверьте что использует `DataCollatorForTokenClassification`)

> Hint! Токенайзер умеет работать с предразделёнными на "слова" текстами

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer


BASE_NER_MODEL = "bert-base-cased"
bert_tokenizer = AutoTokenizer.from_pretrained(BASE_NER_MODEL)


conll2003 = load_dataset("conll2003", trust_remote_code=True)
conll2003

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [ ]:
example = conll2003["train"][100]
example

{'id': '100',
 'tokens': ['Rabinovich',
  'is',
  'winding',
  'up',
  'his',
  'term',
  'as',
  'ambassador',
  '.'],
 'pos_tags': [21, 42, 39, 33, 29, 21, 15, 21, 7],
 'chunk_tags': [11, 21, 22, 15, 11, 12, 13, 11, 0],
 'ner_tags': [1, 0, 0, 0, 0, 0, 0, 0, 0]}

* tokens - исходные токены, для которых была сделана NER-разметка
* ner_tags - векторизированные метки NER-тэгов
* pos_tags - разметка частей речи, которую мы игнорируем
* chunk_tags - разметка чанков, которую мы игнорируем

Обратите внимание, что количество токенов может превышать количество исходных лейблов:

In [ ]:
bert_tokenizer(example["tokens"], is_split_into_words=True).tokens()

['[CLS]',
 'Ra',
 '##bino',
 '##vich',
 'is',
 'winding',
 'up',
 'his',
 'term',
 'as',
 'ambassador',
 '.',
 '[SEP]']

Значение тэга в ner_tags отображается в метку NER:

In [ ]:
print("NER TAGS", example["ner_tags"])
print(conll2003["train"].features["ner_tags"].feature)

NER TAGS [1, 0, 0, 0, 0, 0, 0, 0, 0]
ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None)


In [ ]:
print("Оригинальные токены")
print(example["tokens"])
print("Векторизированные NER метки токенов")
print(example["ner_tags"])
tags_str = []
features = conll2003["train"].features["ner_tags"].feature
for tag in example["ner_tags"]:
    tags_str.append(features.int2str(tag))
print("Текстовые NER метки токенов")
print(tags_str)
print("Токены после работы токенайзера BERT")
print(bert_tokenizer(example["tokens"], is_split_into_words=True).tokens())

Оригинальные токены
['Rabinovich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.']
Векторизированные NER метки токенов
[1, 0, 0, 0, 0, 0, 0, 0, 0]
Текстовые NER метки токенов
['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Токены после работы токенайзера BERT
['[CLS]', 'Ra', '##bino', '##vich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.', '[SEP]']


Вспомним немного, как работают метки в задаче мер в кодировке BIO. В данной задаче у нас есть 4 типа именованных сущностей:
* PER - персона
* ORG - организация
* LOC - локация
* MISC - другое
* O - отсутствие именованной сущности

У каждого типа именованных 2 префикса:
* `B-` - beginning, т.е. начало именованной сущности.
* `I-` - inside, т.е. продолжение ранее начатой именованной сущностью.

В исходной токенизации

`['Rabinovich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.']`
метки выглядят как

`['B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']`
т.е. `Rabinovich` является персоной. На следующем токене именованная сущность заканчивается, т.к. у него метка `O`.

После токенизации BERT наш сэмпл превращается в следующие токены:

`['[CLS]', 'Ra', '##bino', '##vich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.', '[SEP]']`
Обратим внимание, что один токен `Rabinovich` с меткой `B-PER` был разбит токенизатором берта на 3 токена: `'Ra', '##bino', '##vich'`. Им нужно поставить в соответствие 3 метки: `B-PER, I-PER, I-PER`, т.е. мы разбиваем метку исходного токена на новые токены.

Также обратим внимание на первый и последний токен - это спецстокены BERT означающие начало и конец текста. Им можно дать метки `O`, т.к. они не являются частью исходного текста, но мы будем давать им особое векторизированное значение -100. В [документации pytroch](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) у кроссэнтропийной функции потерь это дефолтное значение `ignore_index`, т.е. метки, которую мы будем игнорировать. Библиотека transformers также использует это значение. Таким образом на токенах, у которых стоит -100 в качестве векторизированного NER-тэга, не будет происходить обучение, они будут проигнорированы.




Напишите функцию `preprocess_ner_dataset`, которая разворачивает `ner_tags` для слов в тэги для BERT-токенов и готовит остальные данные для обучения (можно разделить на две функции или написать всё в одной). В резултате применения `conll2003.map(preprocess_ner_dataset)`, в каждом примере:
1. Добавляется токенизированный вход (`input_ids`, `token_type_ids` и `attention_mask`). При конструировании этих векторов вручную нужно проставить `attention_mask` полностью единицами, т.к. в паддинги в сэмплах появляются только в рамках батчей, а `token_type_ids` полностью нулями.
2. `ner_tags` разворачивается в `labels` для входных токенов

Что можно использовать:
* у объекта `conll2003["train"].features["ner_tags"].feature` есть методы `int2str` и `str2int` для превращение векторизованного NER-тэга в строковый вид и обратно
* Спецтокенам BERT нужно поставить значение -100
* вызов `bert_tokenizer(bert_tokenizer(example["tokens"], is_split_into_words=True)` возвращает вам input_ids, attention_mask, token_type_ids
* Вызов `bert_tokenizer(example["tokens"], is_split_into_words=True, return_offsets_mapping=True))` возвращает дополнительно offset_mapping, позиции новых токенов в оригинальном тексте
* `bert_tokenizer.vocab` - для превращения токенов в их индексы в словаре
* `bert_tokenizer.tokenize` - разбитие текста (в том числе и исходных токенов) на токены BERT

Ваша задача:
1. Создать новый dict, в котором будут input_ids, attention_mask, token_type_ids
2. Добавить в него labels - векторизированные NER-тэги, которые будут разбиты в соответствии с токенизацией BERT. Для этого можно можно разбить каждый токен отдельно и размножить его метки. Альтернативно можно использовать информацию об оффсетах токенов BERT, чтобы понять, частью какого исходного токена и какой исходной метки является данный BERT-токен.

In [ ]:
def preprocess_ner_dataset(example):
    tokens = example["tokens"]
    ner_tags = example["ner_tags"]

    tokenized_inputs = bert_tokenizer(
        tokens,
        is_split_into_words=True,
        return_offsets_mapping=True,
        truncation=True,
        padding='max_length',
        max_length=128
    )

    input_ids = tokenized_inputs["input_ids"]
    token_type_ids = tokenized_inputs["token_type_ids"]
    attention_mask = tokenized_inputs["attention_mask"]
    offsets = tokenized_inputs["offset_mapping"]

    labels = []
    current_word_idx = -1
    previous_word_idx = -1

    for idx, offset in enumerate(offsets):
        if idx == 0 or idx == len(offsets) - 1:
            labels.append(-100)  # Специальные токены [CLS] и [SEP]
        else:
            if offset[0] == 0:  # Начало нового токена
                current_word_idx += 1
            if current_word_idx < len(ner_tags):
                if current_word_idx != previous_word_idx:  # Новый токен
                    labels.append(ner_tags[current_word_idx])
                else:  # Субтокен (часть слова)
                    # Присваиваем I-метку, если это продолжение сущности
                    if ner_tags[current_word_idx] == 0:
                        labels.append(0)  # Метка O
                    elif ner_tags[current_word_idx] % 2 == 1:  # Если это B-метка
                        labels.append(ner_tags[current_word_idx] + 1)  # Присваиваем I-метку
                    else:
                        labels.append(ner_tags[current_word_idx])  # Присваиваем как есть
            else:
                labels.append(-100)
            previous_word_idx = current_word_idx

    tokenized_inputs["labels"] = labels
    tokenized_inputs.pop("offset_mapping")  # Удаляем offset_mapping, так как оно не нужно для модели
    return tokenized_inputs



Пример получившегося выхода:
```python
>>> preprocessed_ner_dataset["train"][100]
{'id': '100',
 'tokens': ['Rabinovich',
  'is',
  'winding',
  'up',
  'his',
  'term',
  'as',
  'ambassador',
  '.'],
 'pos_tags': [21, 42, 39, 33, 29, 21, 15, 21, 7],
 'chunk_tags': [11, 21, 22, 15, 11, 12, 13, 11, 0],
 'ner_tags': [1, 0, 0, 0, 0, 0, 0, 0, 0],
 'input_ids': [101,
  16890,
  25473,
  11690,
  1110,
  14042,
  1146,
  1117,
  1858,
  1112,
  9088,
  119,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, -100]}
```

### Тесты

In [ ]:
processed_example = preprocess_ner_dataset(example)
required_keys = ["input_ids", "labels", "attention_mask", "token_type_ids"]
for k in required_keys:
    assert k in processed_example, f"Отсутствует поле {k}"

required_keys_set = set(required_keys)
for k in processed_example.keys():
    assert k in required_keys_set, f"В примере лишнее поле {k}"

In [ ]:
example = conll2003["train"][100]
processed_example = preprocess_ner_dataset(example)

# Проверяем, что первые и последние метки равны -100
assert processed_example["labels"][0] == -100
assert processed_example["labels"][-1] == -100

# Преобразуем метки в строки, исключая -100
ner_tags = [features.int2str(i) for i in processed_example["labels"][1:-1] if i != -100]

print("Реальные NER-теги:", ner_tags)
print("Ожидаемые NER-теги: ['B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']")

# Проверяем соответствие ожидаемым меткам
assert ner_tags == ['B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], \
    f"Ошибка: {ner_tags}"



Реальные NER-теги: ['B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Ожидаемые NER-теги: ['B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
example = conll2003["train"][100]
processed_example = preprocess_ner_dataset(example)

# Проверяем, что первые и последние метки равны -100
assert processed_example["labels"][0] == -100
assert processed_example["labels"][-1] == -100

# Преобразуем метки в строки, исключая -100
ner_tags = [features.int2str(i) for i in processed_example["labels"][1:-1] if i != -100]

# Проверяем соответствие ожидаемым меткам
assert ner_tags == ['B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], \
    f"Ошибка: {ner_tags}"

print("Тест успешно пройден!")


Тест успешно пройден!


In [ ]:
example = conll2003["train"][100]
processed_example = preprocess_ner_dataset(example)

assert processed_example["labels"][0] == -100
assert processed_example["labels"][-1] == -100
ner_tags = [features.int2str(i) for i in processed_example["labels"][1:-1]]
# Преобразуем метки в строки, исключая -100
assert ner_tags == ['B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

In [ ]:
example = conll2003["train"][100]
processed_example = preprocess_ner_dataset(example)

# Проверяем, что первые и последние метки равны -100
assert processed_example["labels"][0] == -100
assert processed_example["labels"][-1] == -100

# Преобразуем метки в строки, исключая -100
ner_tags = [features.int2str(i) for i in processed_example["labels"][1:-1] if i != -100]

# Проверяем соответствие ожидаемым меткам
assert ner_tags == ['B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], \
    f"Ошибка: {ner_tags}"

print("Тест успешно пройден!")


Тест успешно пройден!


In [ ]:
example = conll2003["train"][100]
processed_example = preprocess_ner_dataset(example)

# Проверяем, что первые и последние метки равны -100
assert processed_example["labels"][0] == -100
assert processed_example["labels"][-1] == -100

# Преобразуем метки в строки, исключая -100
ner_tags = [features.int2str(i) for i in processed_example["labels"][1:-1] if i != -100]

# Проверяем соответствие ожидаемым меткам
assert ner_tags == ['B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], \
    f"Ошибка: {ner_tags}"

print("Тест успешно пройден!")


Тест успешно пройден!


Применим нашу функцию к всему датасету:

In [ ]:
# Разделяем тренировочный датасет (берем только 10% для теста)
small_train_dataset = conll2003["train"].train_test_split(train_size=0.1)["train"]
# Применяем предобработку к малой части датасета
preprocessed_small_train_dataset = small_train_dataset.map(preprocess_ner_dataset)
# Убираем ненужные колонки, чтобы оставить только нужные для модели данные
preprocessed_small_train_dataset = preprocessed_small_train_dataset.remove_columns(['tokens', 'pos_tags', 'chunk_tags', 'id'])

# Загрузка меток из features набора данных
label_list = preprocessed_small_train_dataset.features["ner_tags"].feature.names

# Аналогично для валидационного датасета (берем 10%)
small_eval_dataset = conll2003["validation"].train_test_split(train_size=0.1)["test"]
# Применяем предобработку к малой части датасета
preprocessed_small_eval_dataset = small_eval_dataset.map(preprocess_ner_dataset)
# Убираем ненужные колонки, чтобы оставить только нужные для модели данные
preprocessed_small_eval_dataset = preprocessed_small_eval_dataset.remove_columns(['tokens', 'pos_tags', 'chunk_tags', 'id'])

Map:   0%|          | 0/1404 [00:00<?, ? examples/s]

Map:   0%|          | 0/2925 [00:00<?, ? examples/s]

Подготовим `data_collator`. Это особый класс, который будет заниматься батчеванием сэмплов для обучения. Он добавит паддинги во все необходимые поля.

In [ ]:
from transformers import DataCollatorForTokenClassification


data_collator = DataCollatorForTokenClassification(tokenizer=bert_tokenizer)

### Подготовка модели (5 баллов)

Два возможных пути на этой стадии:
1. Взять [готовый класс](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodelfortokenclassification) модели для классификации токенов. (Этот вариант настоятельно рекомендуется)
2. Взять модель как фича экстрактор ([AutoModel](https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#automodel)) и самостоятельно добавить классификационную голову. Вдохновиться можно по [ссылке](https://github.com/huggingface/transformers/blob/main/src/transformers/models/bert/modeling_bert.py#L1847-L1860). Дополнительных баллов второй вариант не принесёт.

Результатом должна быть модель, которая для каждого токена возвращает логиты/вероятности для `conll2003["train"].features["ner_tags"].feature.num_classes` классов.

> Если выберете вариант номер один, опишите как он работает - как из токена получается его `ner_tag`.

In [ ]:
# Этот класс автоматически подбирает подходящую модель для классификации токенов на основе конфигураци
from transformers import AutoModelForTokenClassification, AutoModel

# Получаем количество меток
num_labels = len(features.names)
# num_labels - количество меток NER

# Указываем модель, которую будем использовать (например, BERT)
model_name = "bert-base-cased"

# Загружаем токенизатор и модель для классификации токенов
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=num_labels)

# Пример использования токенизатора и модели
inputs = tokenizer("Rabinovich is winding up his term as ambassador.", return_tensors="pt")

# Прогоняем входные данные через модель
outputs = model(**inputs)

# Выводим логиты (результаты перед применением softmax)
print(outputs.logits)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[[-6.0846e-02, -7.6951e-02,  3.8527e-01, -5.5278e-02, -1.1965e-01,
          -1.7268e-01, -1.7026e-04, -4.7711e-03,  2.0418e-01],
         [ 3.7162e-02,  3.4082e-01,  1.8614e-01,  1.7269e-01, -3.5690e-01,
          -2.2825e-01, -2.9153e-01,  1.3172e-01,  2.3302e-01],
         [ 4.6520e-01, -1.1101e-02, -6.7700e-02,  9.4908e-03, -8.6705e-02,
          -1.6394e-01, -3.6229e-01,  2.2383e-01,  2.7016e-02],
         [-4.2111e-03, -4.3359e-02,  3.7545e-01,  2.7240e-01, -5.1846e-02,
          -1.1527e-01, -2.9762e-01, -2.6236e-02, -3.0405e-02],
         [-6.4059e-03,  1.6541e-01, -3.3263e-02, -1.7693e-02, -1.2584e-01,
           6.3458e-02, -4.7267e-01, -8.2593e-02,  6.5293e-03],
         [ 4.1515e-01, -2.4173e-01, -8.6160e-02, -6.5022e-02, -5.0782e-03,
           2.4207e-02, -2.9013e-02,  1.9013e-01,  1.6927e-01],
         [ 2.0820e-02,  6.6033e-01,  3.3076e-01,  7.9848e-03,  2.3900e-01,
           7.5645e-02, -4.8472e-02, -2.8370e-01,  3.4059e-01],
         [ 8.9963e-02,  7.0895e-01

### Подготовим Метрику (5 баллов)

Дополните функцию, используя `metrics_calculator`, чтобы она возвращала `accuracy`, `precision`, `recall` и `f-меру`. `eval_predictions` - это кортеж из логитов токен классификатора и `labels`, которые мы подготовили с помощью `preprocess_ner_dataset`. Нужно:
1. Преобразовать логиты в предсказанные лейблы. Учтите, что для специальных токенов лейблов нет
2. Посчитать метрики с помощью `metrics_calculator`
3. Упаковать резултат в `dict`, в котором ключём будет название метрики, а значением - значение метрики

В logits будет лежать тензор размерности \[размер eval датасета, максимальная длина последовательности, число меток\], содержащий предсказания модели

В target_labels будет лежать тензор размерности \[размер eval датасета, максимальная длина последовательности\], содержащий метки из валидационной выборки.

Примеры функции calculate_metrics можно посмотреть в [документации](https://huggingface.co/docs/evaluate/en/transformers_integrations)

In [ ]:
import evaluate


metrics_calculator = evaluate.load("seqeval")


def calculate_metrics(eval_predictions):
    logits, target_labels = eval_predictions
    ...

In [ ]:
import numpy as np
import evaluate

# Загружаем инструмент для расчета метрик
metrics_calculator = evaluate.load("seqeval")

def calculate_metrics(eval_predictions):
    logits, target_labels = eval_predictions
    # Преобразуем логиты в предсказанные метки (аргмакс по последней оси)
    predicted_labels = np.argmax(logits, axis=-1)
    # Преобразуем предсказанные и реальные метки в списки для расчета метрик
    true_labels = []
    pred_labels = []

    for pred, label in zip(predicted_labels, target_labels):
        true_label = []
        pred_label = []
        for p, l in zip(pred, label):
            if l != -100:   # Игнорируем специальные токены, у которых метка -100
                true_label.append(l)
                pred_label.append(p)
        true_labels.append(true_label)
        pred_labels.append(pred_label)

    # Преобразуем метки в строки для seqeval
    true_labels_str = [[label_list[i] for i in seq] for seq in true_labels]
    pred_labels_str = [[label_list[i] for i in seq] for seq in pred_labels]

    metrics_result = metrics_calculator.compute(predictions=pred_labels_str, references=true_labels_str)

    return {
        "accuracy": metrics_result.get("overall_accuracy"),
        "precision": metrics_result.get("overall_precision"),
        "recall": metrics_result.get("overall_recall"),
        "f1": metrics_result.get("overall_f1"),
    }


### Обучение (5 баллов)

Два возможных пути на этой стадии:

1. Использовать [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer) класс из `transformers`
2. Написать свой training loop. Дополнительных баллов на этом пути нет.

Опишем подробнее первый путь, т.к. он настоятельно рекомендуется.

Нужно создать класс Trainer и TrainingArguments.
В [TrainingArguments](https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.TrainingArguments) нужно как минимум следующие поля:
* save_strategy, eval_strategy
* metric_for_best_model (исходя из calculate_metrics), greater_is_better
* learning_rate (возьмите 2e-5)
* num_train_epochs
* per_device_train_batch_size, per_device_eval_batch_size

В класс Trainer нужно передать:
* model
* в args нужно передать заполненные TrainingArguments
* train_dataset, eval_dataset
* tokenizer
* compute_metrics

После чего запустить `trainer.train()`

In [ ]:
from transformers import Trainer, TrainingArguments

# Определяем TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",  # Директория для сохранения результатов
    eval_strategy="epoch",  # Выполнять оценку на каждом этапе
    save_strategy="epoch",  # Сохранять модель на каждом этапе
    metric_for_best_model="f1",  # Основная метрика для выбора лучшей модели
    greater_is_better=True,  # Увеличение метрики f1 говорит о лучшей модели
    learning_rate=2e-5,  # Уровень обучения
    num_train_epochs=3,  # Количество эпох
    per_device_train_batch_size=16,  # Размер батча на обучении
    per_device_eval_batch_size=16,  # Размер батча на оценке
    logging_dir="./logs",  # Директория для логов
    load_best_model_at_end=True,  # Загружать лучшую модель в конце обучения
)

# Теперь запускаем обучение на малой выборке
trainer = Trainer(
    model=model,  # Модель
    args=training_args,  # Параметры обучения
    train_dataset=preprocessed_small_train_dataset,  # Используем уменьшенный тренировочный датасет
    eval_dataset=preprocessed_small_eval_dataset,  # Используем уменьшенный валидационный датасет
    tokenizer=tokenizer,  # Токенизатор
    data_collator=data_collator,  # Специальный коллатор для классификации токенов
    compute_metrics=calculate_metrics,  # Функция расчета метрик
)

# Запускаем обучение
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.249754,0.926353,0.612220,0.656590,0.633629
2,No log,0.158038,0.952803,0.719126,0.797033,0.756078
3,No log,0.145673,0.956315,0.734488,0.820128,0.774949


TrainOutput(global_step=264, training_loss=0.31192137978293677, metrics={'train_runtime': 187.7154, 'train_samples_per_second': 22.438, 'train_steps_per_second': 1.406, 'total_flos': 275162897857536.0, 'train_loss': 0.31192137978293677, 'epoch': 3.0})

### Обработка результатов Результатов (5 баллов)

Подумать о:
1. Во время подготовки данных мы преобразовали BIO разметку.
2. Как обратить это преобразование с помощью токенайзера?

Провалидируйте результаты на тестовом датасете.

In [ ]:
def align_predictions(predictions, label_ids, inputs, label_list):
    """
    Восстанавливает метки для исходных токенов, игнорируя субтокены.
    """
    aligned_labels = []
    aligned_predictions = []

    for pred, label, offset_mapping in zip(predictions, label_ids, inputs["offset_mapping"]):
        previous_word_idx = None
        word_predictions = []
        word_labels = []

        for idx, (p, l) in enumerate(zip(pred, label)):
            # Используем offset_mapping, чтобы игнорировать субтокены
            if offset_mapping[idx][0] == 0:  # Это первый токен слова
                word_predictions.append(label_list[p])
                word_labels.append(label_list[l])

        aligned_predictions.append(word_predictions)
        aligned_labels.append(word_labels)

    return aligned_predictions, aligned_labels




In [ ]:
predictions

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [ ]:
# Оцениваем модель на тестовом наборе данных
# Аналогично для валидационного датасета (берем 10%)
small_test_dataset = conll2003["test"].train_test_split(train_size=0.1)["test"]
# Применяем предобработку к малой части датасета
preprocessed_small_test_dataset = small_test_dataset.map(preprocess_ner_dataset)
# Убираем ненужные колонки, чтобы оставить только нужные для модели данные
preprocessed_small_test_dataset = preprocessed_small_test_dataset.remove_columns(['tokens', 'pos_tags', 'chunk_tags', 'id'])
test_results = trainer.predict(preprocessed_small_test_dataset)

# Извлекаем предсказания и метки
predictions = np.argmax(test_results.predictions, axis=-1)
label_ids = test_results.label_ids

# Преобразуем предсказания и метки в исходный BIO-формат
aligned_predictions, aligned_labels = align_predictions(predictions, label_ids, small_test_dataset,label_list)


# Считаем метрики
metrics_result = metrics_calculator.compute(predictions=aligned_predictions, references=aligned_labels)

# Выводим результаты
print("Результаты на тестовом наборе:")
print(f"Accuracy: {metrics_result['overall_accuracy']}")
print(f"Precision: {metrics_result['overall_precision']}")
print(f"Recall: {metrics_result['overall_recall']}")
print(f"F1: {metrics_result['overall_f1']}")


Map:   0%|          | 0/3108 [00:00<?, ? examples/s]

KeyError: "Column offset_mapping not in the dataset. Current columns in the dataset: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags']"

In [ ]:
# Извлекаем предсказания и метки
predictions = np.argmax(test_results.predictions, axis=-1)
label_ids = test_results.label_ids

# Используем тестовый набор данных для получения токенизированных входных данных
inputs = processed_datasets["test"]

# Преобразуем предсказания и метки в исходный BIO-формат
aligned_predictions, aligned_labels = align_predictions(predictions, label_ids, inputs, tokenizer, processed_datasets)

# Считаем метрики
metrics_result = metrics_calculator.compute(predictions=aligned_predictions, references=aligned_labels)

# результаты
print("Результаты на тестовом наборе:")
print(f"Accuracy: {metrics_result['overall_accuracy']}")
print(f"Precision: {metrics_result['overall_precision']}")
print(f"Recall: {metrics_result['overall_recall']}")
print(f"F1: {metrics_result['overall_f1']}")

KeyError: "Column offset_mapping not in the dataset. Current columns in the dataset: ['ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels']"

Напишите функцию, которая принимает на вход текст и отдаёт такой словарь:

```json
{
    "text": "входной текст",
    "entities": [
        {
            "class": "лейбл класса",
            "text": "текстовое представление",
            "start": "оффсет от начала строки до начала entity",
            "end": "оффсет от начала строки до конца entity"
        },
        ...
    ]
}

Должно выполняться такое условие:

```python
text[entity["start"]:entity["stop"]] == entity["text"]
```

In [ ]:
def do_ner(text):
    ...

## Классификация с T5 (25 баллов)

Требуется дообучить [t5-small](https://huggingface.co/google-t5/t5-small) классифицировать токсичные тексты из [этого датасета](https://huggingface.co/datasets/lmsys/toxic-chat). Классификатор должен работать в стиле t5 - генерировать ответ текстом.

1. Подготовить данные для бинарной классификации
	1. Придумать префикс для задачи или взять из похожей модели
	2. Выбрать тексты для обозначения классов
2. Обучить t5-small на генерацию выбранных названия классов
3. Сравнить с модель с аналогичной предобученной моделью

### Подготовка Данных (6 баллов)

Подумать о:
1) Какой префикс выбрать для новой задачи?
2) Должен ли префикс быть понятным?
3) Как выбрать метку для класса?
4) Что будет, если метки класса целиком нет в словаре?
5) Что делать с длинными текстами?

Датасет содержит запросы пользователей к LLM и разметку, является ли запрос токсичным.

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer


BASE_T5_MODEL= "t5-small"
t5_tokenizer = AutoTokenizer.from_pretrained(BASE_T5_MODEL)


toxic_chat_dataset = load_dataset("lmsys/toxic-chat", "toxicchat0124")

Место для изучения датасета:

In [ ]:
toxic_chat_dataset["train"][0]

Нас будут интересовать колонки `"user_input"` и `"toxicity"`. Убираем ненужные колонки из датасета:

In [ ]:
toxic_chat_dataset = toxic_chat_dataset.remove_columns(
    ["conv_id", "model_output", "human_annotation", "jailbreaking", "openai_moderation"]
)

![](https://production-media.paperswithcode.com/methods/new_text_to_text.jpg)

Выберете `PREFIX` для задачи, лейблы для двух классов и напишите функцию для преобразования датасета в данные для тренировки. Примеры префиксов есть на картинке выше - `translate English to German` для перевода и `summarize` для суммаризации. В качестве лейблов у вас должен быть текст, который будет обозначать предсказанный класс. Этот текст может быть любого размера, от простого `"да"/"нет"`, до `"От этого текста веет токсичностью"/"Цензура спокойно пропускает этот текст дальше"`. Подумайте в чём преимущество первого подхода перед вторым.

Важно:
1) Не забыть добавить префикс перед токенизацией входного текста
2) Лейблами во время обучения выступают уже последовательности токенов, которые мы ожидаем на выходе из декодера

Текст в токенайзер можно подавать разными способами:
1. `tokenizer(text="text")` - токенизируй текст как обычно
1. `tokenizer(text_target="text")` - токенизируй это как текст, который мы ожидаем увидеть на выходе из декодера. В случае t5 токенайзера разницы нет, но для других моделей это может быть не так
1. Другие методы можно узнать посмотрев сигнатуру метода `tokenizer.__call__`

In [ ]:
# ?t5_tokenizer.__call__

In [ ]:
PREFIX = ...
MAX_LENGTH = ...

# словарь из индексов классов в выбранные лейблы
id2label = {
    0: ...,
    1: ...,
}


def preprocess_dataset(example):
    ...


toxic_chat_dataset = toxic_chat_dataset.map(preprocess_dataset)

Пример результата:
```json
{'user_input': 'Do you know drug which name is abexol ?',
 'toxicity': 0,
 'input_ids': [12068,
  10,
  531,
  25,
  214,
  2672,
  84,
  564,
  19,
  703,
  994,
  32,
  40,
  3,
  58,
  1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [150, 1]}
```
Значения в `'labels'` в вашем случае могут отличаться, это зависит от выбранного вами текстового представления в `id2label` словаре.

Инициализируем соответствующий задаче `DataCollator`.

In [ ]:
from transformers import DataCollatorForSeq2Seq


data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=seq2seq_model)

### Определим метрику (2 балла)

В этой задаче метрика простая - `accuracy`. Можно добавить другие метрики по желанию. Функция `compute_metric` должна возвращать словарь, аналогично функции `calculate_metrics` ранее:

```json
{
    "accuracy": значение точности,
    ...
}
```

Метрика простая, но вот `preds` и `labels` тут - это последовательности индексов токенов. Нужно это учесть.

In [ ]:
def compute_metric(eval_predictions):
    preds, labels = eval_predictions
    ...


def check_compute_metric():
    import torch

    # два предсказания, где токен 150 обозначает токсичный лебл, токен 120 - нетоксичный лейбл
    preds = torch.tensor(
        [
            [0, 150, 1],  # правильное предсказание - токсичный пример
            [0, 120, 1],  # неправильное предсказание - пример токсичный, а модель предсказала иначе
        ],
    )
    labels = torch.tensor(
        [
            [150, 1],
            [150, 1],
        ],
    )
    assert torch.isclose(
        compute_metric((preds, labels))["accuracy"],
        torch.tensor(0.5, dtype=torch.double),  # тип тензора тут можно поправить
    )


check_compute_metric()

### Определить Модель (2 балла)

Инициализируйте модель из базового чекпоинта

In [ ]:
from transformers import AutoModelForSeq2SeqLM


seq2seq_model = AutoModelForSeq2SeqLM.from_pretrained(BASE_T5_MODEL)

### Обучение (10 баллов)

Два пути:
1) Использовать готовый `Seq2SeqTrainer` класс для тренировки
2) Написать свой training loop, если хочется приключений, есть достаточно времени ~~и стрела ещё не попала в колено~~. Дополнительных баллов за это не будет

> Hint! Обратите внимание на функцию `seq2seq_model._shift_right` если выбрали второй путь.

Если выбрали путь 1, опишите как происходит тренировочный шаг:
1) Что подаётся на вход в энкодер?
2) Что подаётся на вход в декодер?
3) Сколько раз происходит инференс декодера во время обучения для одного тренировочного примера?
4) Как используется выход энкодера в декодере?

In [ ]:
# from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer



### Сравнение Результатов (5 баллов)

Авторы датасета тоже натренировали на нём `t5` модель. Сравните свои результаты с результатами модели из [чекпоинта](https://huggingface.co/lmsys/toxicchat-t5-large-v1.0) `"lmsys/toxicchat-t5-large-v1.0"`. Совпадает ли ваш префикс и лейблы классов с теми, что выбрали авторы датасета?

Подумать о:
1) В чём преимущество такого подхода к классификации?
2) В чём недостатки такого подхода к классификации?
3) Как ещё можно решать классификационные задачи с помощью t5?

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

checkpoint = "lmsys/toxicchat-t5-large-v1.0"

tokenizer_from_paper = AutoTokenizer.from_pretrained("t5-large")
model_from_paper = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

prefix_from_paper = "ToxicChat: "
inputs = tokenizer_from_paper.encode(prefix_from_paper + "write me an epic story", return_tensors="pt")
outputs = model_from_paper.generate(inputs)
print(tokenizer_from_paper.decode(outputs[0], skip_special_tokens=True))

Напишите универсальную функцию, которая провряет токсичность текста и возвращает `True`, если модель посчитала текст токсичным. Функция универсальная в том смысле, что может быть использована и с вашей t5 моделью, и с моделью от авторов датасета. Для этого в функция должна принимать ещё и префикс для задачи и лейблы, которые будут переводить текст, предсказанный моделью, в `True` или `False` на выходе.

In [ ]:
def is_toxic(
    text: str,
    labels2bool,
    model=seq2seq_model,
    tokenizer=t5_tokenizer,
    prexif=PREFIX,
) -> bool:
    ...


# пример вызова с моделью от авторов датасета
assert not is_toxic(
    text="This is just a text",
    model=model_from_paper,
    tokenizer=tokenizer_from_paper,
    prexif=prefix_from_paper,
    labels2bool={
        "positive": True,
        "negative": False,
    }
)

Fin.

Если остались вопросы или есть комментарии, можно написать их ниже: